## NEXT2.0 Notebook

In [1]:
import os
import datetime
print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

import time
import glob
import tables as tb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from functools import reduce
import numpy.testing as npt
from operator import itemgetter, attrgetter
%matplotlib inline
%load_ext autoreload
%autoreload 2

plt.rcParams["figure.figsize"] = 10, 8
plt.rcParams["font.size"     ] = 14

2017-12-06 14:52:42


In [2]:
from math import sqrt, pi

In [3]:
from IPython.display import Image

In [49]:
import pynext.Next100 as NN
import pynext.Material as mat
import pynext.Shapes as shp
from   pynext.system_of_units import *
import pynext.TpcEL as tpcel

In [69]:
def Next100PV():
    L=130*cm
    R=52*cm
    ts=1*cm
    tc=10*cm
    Rin=R
    Rout=Rin+ts

    nxt = shp.CylinderShell(Rin,Rout,L)
    print("""
    Next PV: = {}
    """.format(nxt))

    nplt = shp.FlatPlate(R,tc)

    print( """
    Next EndCups = {}
    """.format(nplt))
    
    s= """
        Dimensions: (Rin = %7.2f cm, Rout = %7.2f cm, L = %7.2f cm)
        Inner Volume  = %7.2f m3  Shell Volume = %7.2e m3
        Inner Surface = %7.2f m2; Shell Surface = %7.2f m2
        """%(nxt.Rin / cm,
             nxt.Rout / cm,
             nxt.L / cm,
             nxt.InnerVolume  / m3,
             nxt.ShellVolume  / m3,
             nxt.InnerSurface / m2,
             nxt.ShellSurface / m2
             )
    print(s)
    
    
    s= """
        Disk(R = %7.2f m, t = %7.2f cm, S = %7.2f m2, V = %7.2e m3)
        """%(nplt.R / m,
             nplt.t / cm,
             nplt.S / m2,
             nplt.V / m3
             )
    print(s)

In [70]:
Next100PV()


    Next PV: = 
        CylinderShell(Rin = 5.20e+02, Rout = 5.30e+02, L = 1.30e+03)
        Inner Volume  = 1.10e+09; Shell Volume = 4.29e+07
        Inner Surface = 4.25e+06; Shell Surface = 4.33e+06
        
    

    Next EndCups = 
        Disk(R = 5.20e+02, t = 1.00e+02, S = 8.49e+05, V = 8.49e+07)
        
    

        Dimensions: (Rin =   52.00 cm, Rout =   53.00 cm, L =  130.00 cm)
        Inner Volume  =    1.10 m3  Shell Volume = 4.29e-02 m3
        Inner Surface =    4.25 m2; Shell Surface =    4.33 m2
        

        Disk(R =    0.52 m, t =   10.00 cm, S =    0.85 m2, V = 8.49e-02 m3)
        


### Rejection factors etc

In [5]:
Bi214_RF={0.5:5.6e-7,1.0:1.7e-6,2.0:2.1e-6,3.0:2.5e-6,4.0:2.7e-6}
Tl208_RF={0.5:6.1e-7,1.0:1.7e-6,2.0:4.3e-6,3.0:1.4e-5,4.0:4.5e-5}
Qbb_Xe136 =2462.0*keV
TF= 1./10.
R={0.5:0.03,0.9:0.09}

### NEW

#### NEW dimensions

In [6]:
D_NEW_FC = 50 * cm
R_NEW_FC = D_NEW_FC / 2 
L_NEW_FC = 50 * cm
S_NEW = pi*(R_NEW_FC)**2

D_NEW_PV = 50 * cm
R_NEW_PV = D_NEW_PV / 2
t_NEW_PV = 2 * cm 
L_NEW_PV = 80 * cm
S_NEW = pi*(R_NEW_FC)**2

print('NEW: radius = {} cm;  drift = {} cm; sensor surface = {} cm2'.format(
      R_NEW_FC / cm, L_NEW_FC / cm, S_NEW  / cm2))
print('NEW PV: radius = {} cm;  thickness = {} cm; length = {} cm'.format(
      R_NEW_PV / cm, t_NEW_PV / cm, L_NEW_PV  / cm))

NEW: radius = 25.0 cm;  drift = 50.0 cm; sensor surface = 1963.4954084936207 cm2
NEW PV: radius = 25.0 cm;  thickness = 2.0 cm; length = 80.0 cm


#### NEW gas

In [7]:
xeNEW =NN.AngelXe(R = R_NEW_FC, L = L_NEW_FC , Rb=1*cm, Cb=10*cm, Ab=1*cm, P=15)
print(xeNEW)


        Pressure =  15.00 bar
        Fiducial Radius =   25.00 cm  xe Radius =   26.00 cm
        Fiducial Length =   50.00 cm  Xe Length =   61.00 cm
        
        Fiducial Volume =    0.10 m3 Xe Volume =    0.13 m3
        Fiducial mass =    8.48 kg Xe mass =   11.19 kg
        
        Radial buffer radius =    1.00 cm (E/P ~ 2)
        Cathode buffer thickness =   10.00 cm   (E/P~1)
        Anode buffer thickness =    1.00 cm
        
        Long buffer Volume =    0.01 m3
        Long buffer mass =    0.84 kg
        
        Cathode buffer Volume =    0.02 m3
        Cathode buffer mass =    1.84 kg
    
        Anode buffer Volume =    0.00 m3
        Anode buffer mass =    0.18 kg
    
        Total buffer Mass =    2.86 kg
        Cathode Readout Surface =    0.21 m2
    


#### NEW can

In [8]:
canNEW = NN.AngelCan(Rin=R_NEW_PV, Rout=R_NEW_PV + t_NEW_PV, L=L_NEW_PV, Cp=t_NEW_PV, Ap=t_NEW_PV,
                 cplateType='Flat', aplateType='Flat',
                 FlangeRin=R_NEW_PV, FlangeRout=R_NEW_PV + t_NEW_PV, FlangeT=5*cm, Mat='Fe')
print(canNEW)


        Material = Fe
        Inner Radius =   25.00 cm
        Outer Radius =   27.00 cm
        Length =   80.00 cm
        Cathode plate thickness =    2.00 cm
        Anode plate thickness =    2.00 cm
        Flange thickness =    5.00 cm
        
        Shell Volume =    0.03 m3
        Shell mass =  205.71 kg
        Shell activity =   1e+07 c/year
        
        Cathode plate Volume =    0.00 m3
        Cathode plate mass =   36.05 kg
        Cathode plate activity =   2e+06 c/year
        
        Anode plate Volume =    0.00 m3
        Anode plate mass =   36.05 kg
        Anode plate activity =   2e+06 c/year
        
        Flange Volume =    0.00 m3
        Flange mass =   12.86 kg
        Flange activity =   1e+06 c/year
        
        Can total mass =  277.80 kg
        Can total activity = 1.28e+07 counts/year
        


#### NEW EL

In [18]:
newEL = NN.AngelEL(EP=1.3*kilovolt/(cm*bar),
                 dV=0.4*kilovolt/cm,
                 P=15*bar,
                 d=6*mm,
                 L=L_NEW_FC,
                 EPmin=1.5*kilovolt/(cm*bar))
print(newEL)


        E/P =    1.30 kV * cm^-1* bar^-1
        E/P min =    1.50 kV * cm^-1* bar^-1
        dV = drift voltage =    0.40 kV * cm^-1
        P  = pressure =   15.00 bar
        d  = EL grid gap =    6.00 mm
        L  = drift lenght =   0.50 m
        Grid voltage =   11.70 kV
        Cathode voltage =   31.70 kV
        Yield =  5.94e+02 photons/e
        Yield at EPmin =  8.46e+02 photons/e
        Buffer length =     1.4 cm
        dE/dx =  2.21e+01 electrons/cm
        dG/dx =  1.31e+04 photons/cm
    
        Primary scintillation photons per MeV = 1.3158e+04
        Primary ionization electrons per MeV = 4.0323e+04
        


#### NEW energy plane

In [10]:
r1141=NN.PMT(name='R1141',gain=1e+6,D=3*2.5*cm, serFWHM=25*ns,filterRT=50*ohm*200*pF,ampliGain=10,
          QE=0.3,AU=3*becquerel*1e-3,ATh=2*becquerel*1e-3)

print(r1141)

newEP =NN.AngelPMT(r1141,xeNEW,coverage=0.35,n1=1.5,n2=1.6)
print(newEP)


        PMT name = R1141
        Gain = 1.00e+06
        I single electron response iSER =    0.00 milliampere
        V single electron response vSER =    0.10 millivolt
        filter raise time = 1.00e+01 nanoseconds
        V single electron response after RC filter =    0.24 millivolt
        V single electron response after RC filter (amplified) =    2.40 millivolt
        PMT Diameter =    7.50 cm
        PMT Surface =   44.18 cm2
        PMT QE =    0.30
        U activity =    3.00 mBq
        Th activity =    2.00 mBq
        

        PMT name = R1141
        PMT QE =    0.30
        PMT Diameter =    7.50 cm
        PMT Surface =   44.18 cm2
        PMT Window refraction indez =    1.50
        Optical Window refraction indez =    1.60
        Optical Transmittance =    0.59
        nof PMT (coverage=   0.35) =      17
        
        Activity = 1.58e+05 c/year
        


#### NEW tracking plane

In [11]:
sipm = NN.SiPM(name='Hamamatsu',L=1.0*mm, QE=0.5,TPB=0.5, AU=3*becquerel*1e-7,ATh=3*becquerel*1e-7)
print(sipm)

newTP= NN.AngelSiPM(sipm,newEL,Surface=pi*(46 * cm/2)**2,pitch=1.0*cm)
print(newTP)


        SiPM name = Hamamatsu
        SiPM size =    1.00 mm
        SiPM Surface =    1.00 mm2
        SiPM QE =    0.50
        SiPM TPB eff =    0.50
        SiPM global PDE =    0.25
        U activity =    0.30 muBq
        Th activity =    0.30 muBq
        

        SiPM QE =    0.50
        SiPM PDE =    0.25
        SiPM size =    1.00 mm
        SiPM Pitch =   10.00 mm
        Linear Packing factor =    0.10
        Detection Surface =    0.17 m2
        SiPM Surface =  0.0017 m2
        Coverage =    0.01
        nof SiPM = 1.7e+03
        nof ASIC =      26
        nof PDM =      66
        


##### NEW object

In [12]:
new = NN.ANGEL(xeNEW,canNEW,newEP,newEL,newTP)
print(new)


        Optical Gain = 1.05e+03
        Solid Angle  = 2.28e-02
        ScintillationSolidAngle = 2.28e-01
        ELSolidAngle = 2.28e-02
        Scintillation Photons/MeV = 1.32e+04
        Scintillation Reaching Cathode/MeV = 2.99e+03
        Scintillation Photoelectrons/MeV = 1.57e+02
                   EL Photons/MeV = 4.25e+07
                   EL Reaching Cathode/MeV = 9.66e+05
                   EL Photoelectrons/MeV = 2.20e+05
                   EL pe/electron = 1.26e+00

        Background rate (c/(keV*kg*year) = 1.05e-02
        


#### Number of photons and pes for Kr

In [13]:
new.ScintillationReachingCathode(40 * keV)

119.78953148676365

In [14]:
new.ScintillationPhotoelectrons(40 * keV)

6.282406229898946

In [15]:
new.ELReachingCathode(40 * keV)

38655.308975091604

In [16]:
new.ELPhotoelectrons(40 * keV)

8797.952568044395

In [17]:
new.ELPhotoelectrons(5 * keV) / 78000

0.014099282961609607

In [18]:
new.ELPhotoelectrons(2500 * keV) /78000

7.049641480804803

### NEXT

In [19]:
D_NEXT_FC = 105 * cm
R_NEXT_FC = D_NEXT_FC / 2 
L_NEXT_FC = 130 * cm
S_NEXT = pi*(R_NEXT_FC)**2

D_NEXT_PV = 120 * cm
R_NEXT_PV = D_NEXT_PV / 2
t_NEXT_PV = 2 * cm 
L_NEXT_PV = 150 * cm
S_NEXT = pi*(R_NEXT_FC)**2

print('NEW: radius = {} cm;  drift = {} cm; sensor surface = {} cm2'.format(
      R_NEXT_FC / cm, L_NEXT_FC / cm, S_NEXT  / cm2))
print('NEW PV: radius = {} cm;  thickness = {} cm; length = {} cm'.format(
      R_NEXT_PV / cm, t_NEXT_PV / cm, L_NEXT_PV  / cm))

NEW: radius = 52.5 cm;  drift = 130.0 cm; sensor surface = 8659.014751456867 cm2
NEW PV: radius = 60.0 cm;  thickness = 2.0 cm; length = 150.0 cm


#### NEXT gas

In [20]:
xeNEXT =NN.AngelXe(R = R_NEXT_FC, L = L_NEXT_FC , Rb=1*cm, Cb=10*cm, Ab=1*cm, P=30)
print(xeNEXT)


        Pressure =  30.00 bar
        Fiducial Radius =   52.50 cm  xe Radius =   53.50 cm
        Fiducial Length =  130.00 cm  Xe Length =  141.00 cm
        
        Fiducial Volume =    1.13 m3 Xe Volume =    1.27 m3
        Fiducial mass =  194.55 kg Xe mass =  219.13 kg
        
        Radial buffer radius =    1.00 cm (E/P ~ 2)
        Cathode buffer thickness =   10.00 cm   (E/P~1)
        Anode buffer thickness =    1.00 cm
        
        Long buffer Volume =    0.05 m3
        Long buffer mass =    8.12 kg
        
        Cathode buffer Volume =    0.09 m3
        Cathode buffer mass =   15.54 kg
    
        Anode buffer Volume =    0.01 m3
        Anode buffer mass =    1.55 kg
    
        Total buffer Mass =   25.21 kg
        Cathode Readout Surface =    0.90 m2
    


#### NEXT EL

In [21]:
elNEXT = NN.AngelEL(EP=1.8*kilovolt/(cm*bar),
                 dV=0.3*kilovolt/cm,
                 P=15*bar,
                 d=5*mm,
                 L=L_NEXT_FC,
                 EPmin=1.5*kilovolt/(cm*bar))
print(elNEXT)


        E/P =    1.80 kV * cm^-1* bar^-1
        E/P min =    1.50 kV * cm^-1* bar^-1
        dV = drift voltage =    0.30 kV * cm^-1
        P  = pressure =   15.00 bar
        d  = EL grid gap =    5.00 mm
        L  = drift lenght =   1.30 m
        Grid voltage =   13.50 kV
        Cathode voltage =   52.50 kV
        Yield =  1.02e+03 photons/e
        Yield at EPmin =  7.05e+02 photons/e
        Buffer length =     2.3 cm
        dE/dx =  2.21e+01 electrons/cm
        dG/dx =  2.25e+04 photons/cm
    
        Primary scintillation photons per MeV = 1.3158e+04
        Primary ionization electrons per MeV = 4.0323e+04
        


#### NEXT energy plane

In [22]:
sipm_EP = NN.SiPM(name='Hamamatsu',L=10*mm, QE=0.5,TPB=0.5, AU=3*becquerel*1e-6,ATh=3*becquerel*1e-6)
print(sipm_EP)


        SiPM name = Hamamatsu
        SiPM size =   10.00 mm
        SiPM Surface =  100.00 mm2
        SiPM QE =    0.50
        SiPM TPB eff =    0.50
        SiPM global PDE =    0.25
        U activity =    3.00 muBq
        Th activity =    3.00 muBq
        


In [23]:
epNEXT= NN.AngelSiPM(sipm_EP, elNEXT, Surface=S_NEXT, pitch=11*mm)
print(epNEXT)


        SiPM QE =    0.50
        SiPM PDE =    0.25
        SiPM size =   10.00 mm
        SiPM Pitch =   11.00 mm
        Linear Packing factor =    0.91
        Detection Surface =    0.87 m2
        SiPM Surface =    0.72 m2
        Coverage =    0.83
        nof SiPM = 7.2e+03
        nof ASIC = 1.1e+02
        nof PDM = 2.9e+02
        


In [24]:
sipm_TP = NN.SiPM(name='Hamamatsu',L=1*mm, QE=0.5,TPB=0.5, AU=3*becquerel*1e-7,ATh=3*becquerel*1e-7)
print(sipm_TP)


        SiPM name = Hamamatsu
        SiPM size =    1.00 mm
        SiPM Surface =    1.00 mm2
        SiPM QE =    0.50
        SiPM TPB eff =    0.50
        SiPM global PDE =    0.25
        U activity =    0.30 muBq
        Th activity =    0.30 muBq
        


In [25]:
tpNEXT= NN.AngelSiPM(sipm_TP, elNEXT, Surface=S_NEXT, pitch=11*mm)
print(tpNEXT)


        SiPM QE =    0.50
        SiPM PDE =    0.25
        SiPM size =    1.00 mm
        SiPM Pitch =   11.00 mm
        Linear Packing factor =    0.09
        Detection Surface =    0.87 m2
        SiPM Surface =  0.0072 m2
        Coverage =  0.0083
        nof SiPM = 7.2e+03
        nof ASIC = 1.1e+02
        nof PDM = 2.9e+02
        


In [26]:
lar_to_gxe = 1.5/0.125

In [27]:
print('liquid argon to xenon factor = {}'.format(lar_to_gxe))

liquid argon to xenon factor = 12.0


### NEXT-200

#### Dimensions

In [67]:
PN2 = 30
D_N2_FC = 105 * cm
R_N2_FC = D_N2_FC / 2 
L_N2_FC = 150 * cm
S_N2    = pi*(R_N2_FC)**2

D_N2_PV = 110 * cm
R_N2_PV = D_N2_PV / 2
t_N2_PV = 5 * cm 
L_N2_PV = 160 * cm
S_N2 = pi*(R_N2_FC)**2

print('NEXT200: radius = {} cm;  drift = {} cm; sensor surface = {} m2'.format(
      R_N2_FC / cm, L_N2_FC / cm, S_N2  / m2))

NEXT200: radius = 52.5 cm;  drift = 150.0 cm; sensor surface = 0.8659014751456867 m2


#### Xenon

In [68]:
xeN2 =NN.AngelXe(R=R_N2_FC, L=L_N2_FC, Rb=1*cm, Cb=2*cm, Ab=1*cm, P=PN2)
print(xeN2)


        Pressure =  30.00 bar
        Fiducial Radius =   52.50 cm  xe Radius =   53.50 cm
        Fiducial Length =  150.00 cm  Xe Length =  153.00 cm
        
        Fiducial Volume =    1.30 m3 Xe Volume =    1.38 m3
        Fiducial mass =  224.48 kg Xe mass =  237.78 kg
        
        Radial buffer radius =    1.00 cm (E/P ~ 2)
        Cathode buffer thickness =    2.00 cm   (E/P~1)
        Anode buffer thickness =    1.00 cm
        
        Long buffer Volume =    0.05 m3
        Long buffer mass =    8.81 kg
        
        Cathode buffer Volume =    0.02 m3
        Cathode buffer mass =    3.11 kg
    
        Anode buffer Volume =    0.01 m3
        Anode buffer mass =    1.55 kg
    
        Total buffer Mass =   13.47 kg
        Cathode Readout Surface =    0.90 m2
    


#### Can

In [69]:
canN2 = NN.AngelCan(Rin=R_N2_PV, Rout=R_N2_PV + R_N2_PV , L=L_N2_PV, Cp=L_N2_PV, Ap=L_N2_PV,
                 cplateType='Flat', aplateType='Flat',
                 FlangeRin=R_N2_PV, FlangeRout=R_N2_PV + t_N2_PV, FlangeT=R_N2_PV + t_N2_PV, Mat='Fe')
print(canN2)


        Material = Fe
        Inner Radius =   55.00 cm
        Outer Radius =  110.00 cm
        Length =  160.00 cm
        Cathode plate thickness =  160.00 cm
        Anode plate thickness =  160.00 cm
        Flange thickness =   60.00 cm
        
        Shell Volume =    4.56 m3
        Shell mass = 35899.73 kg
        Shell activity =   2e+08 c/year
        
        Cathode plate Volume =    6.08 m3
        Cathode plate mass = 47866.31 kg
        Cathode plate activity =   6e+07 c/year
        
        Anode plate Volume =    6.08 m3
        Anode plate mass = 47866.31 kg
        Anode plate activity =   6e+07 c/year
        
        Flange Volume =    0.11 m3
        Flange mass =  852.99 kg
        Flange activity =   3e+07 c/year
        
        Can total mass = 131632.36 kg
        Can total activity = 2.87e+08 counts/year
        


#### EL

In [70]:
elN2 = NN.AngelEL(EP=1.5*kilovolt/(cm*bar),
                 dV=0.3*kilovolt/cm,
                 P=PN2*bar,
                 d=5*mm,
                 L=L_N2_FC / 2,
                 EPmin=1.0*kilovolt/(cm*bar))
print(elN2)


        E/P =    1.50 kV * cm^-1* bar^-1
        E/P min =    1.00 kV * cm^-1* bar^-1
        dV = drift voltage =    0.30 kV * cm^-1
        P  = pressure =   30.00 bar
        d  = EL grid gap =    5.00 mm
        L  = drift lenght =   0.75 m
        Grid voltage =   22.50 kV
        Cathode voltage =   45.00 kV
        Yield =  1.41e+03 photons/e
        Yield at EPmin =  3.60e+02 photons/e
        Buffer length =     1.5 cm
        dE/dx =  1.10e+01 electrons/cm
        dG/dx =  1.56e+04 photons/cm
    
        Primary scintillation photons per MeV = 1.3158e+04
        Primary ionization electrons per MeV = 4.0323e+04
        


#### Energy plane

In [71]:
sipm_EP = NN.SiPM(name='Hamamatsu',L=10*mm, QE=0.5,TPB=0.5, AU=3*becquerel*1e-9,ATh=3*becquerel*1e-9)
print(sipm_EP)
ep_N2= NN.AngelSiPM(sipm_EP, elN2, Surface=S_N2, pitch=11*mm, asic=64, pdm=25)
print(ep_N2)


        SiPM name = Hamamatsu
        SiPM size =   10.00 mm
        SiPM Surface =  100.00 mm2
        SiPM QE =    0.50
        SiPM TPB eff =    0.50
        SiPM global PDE =    0.25
        U activity =    0.00 muBq
        Th activity =    0.00 muBq
        

        SiPM QE =    0.50
        SiPM PDE =    0.25
        SiPM size =   10.00 mm
        SiPM Pitch =   11.00 mm
        Linear Packing factor =    0.91
        Detection Surface =    0.87 m2
        SiPM Surface =    0.72 m2
        Coverage =    0.83
        nof SiPM = 7.2e+03
        nof ASIC = 1.1e+02
        nof PDM = 2.9e+02
        


#### Tracking plane

In [72]:
sipm_TP = NN.SiPM(name='Hamamatsu',L=1*mm, QE=0.5,TPB=0.5, AU=3*becquerel*1e-7,ATh=3*becquerel*1e-7)
print(sipm_TP)


        SiPM name = Hamamatsu
        SiPM size =    1.00 mm
        SiPM Surface =    1.00 mm2
        SiPM QE =    0.50
        SiPM TPB eff =    0.50
        SiPM global PDE =    0.25
        U activity =    0.30 muBq
        Th activity =    0.30 muBq
        


In [73]:
tp_N2= NN.AngelSiPM(sipm_TP, elN2, Surface=S_N2, pitch=10*mm, asic=64, pdm=1)
print(tp_N2)


        SiPM QE =    0.50
        SiPM PDE =    0.25
        SiPM size =    1.00 mm
        SiPM Pitch =   10.00 mm
        Linear Packing factor =    0.10
        Detection Surface =    0.87 m2
        SiPM Surface =  0.0087 m2
        Coverage =    0.01
        nof SiPM = 8.7e+03
        nof ASIC = 1.4e+02
        nof PDM = 8.7e+03
        


In [75]:
N200 = NN.NausicaAngel(xeN2,canN2,elN2,ep_N2,tp_N2)
print(N200)


        Optical Gain = 1.41e+03
        Solid Angle  = 2.49e-02
        ScintillationSolidAngle = 2.49e-01
        ELSolidAngle = 2.49e-02
        Scintillation Photons/MeV = 1.32e+04
        Scintillation Reaching Cathode/MeV = 3.28e+03
        Scintillation Photoelectrons/MeV = 6.78e+02
                   EL Photons/MeV = 5.69e+07
                   EL Reaching Cathode/MeV = 1.42e+06
                   EL Photoelectrons/MeV = 2.93e+05
                   EL pe/electron = 7.26e+00

        Background rate (c/(keV*kg*year) = 8.82e-03
        


In [76]:
print(N200.ScintillationReachingCathode(40 * keV))
print(N200.ScintillationPhotoelectrons(40 * keV))
print(N200.ScintillationPhotoelectrons(40 * keV) / nof_esipm)
print(N200.ELReachingCathode(40 * keV))
print(N200.ELPhotoelectrons(40 * keV))
print(N200.ELPhotoelectrons(40 * keV) / nof_esipm)
print(N200.ScintillationPhotoelectrons(2500 * keV))
print(N200.ScintillationPhotoelectrons(2500 * keV) / nof_esipm)
print(N200.ELPhotoelectrons(2500 * keV) /nof_esipm)

131.1645682421022
27.10011740539301
0.003763905195193474
56675.786825901894
11709.873311136753
1.626371293213438
1693.757337837063
0.23524407469959208
101.64820582583988


### NEXT2.0

#### gas dimensions

In [77]:
P = 10
D_N2T_FC = 300 * cm
L_N2T_FC = 300 * cm
t_gas_buffer = 1 * cm
n2tf = shp.Cylinder(D_N2T_FC/2, L_N2T_FC)

print('NEXT2.0: radius = {} cm;  drift = {} cm; sensor surface = {} m2'.format(
     n2tf.R / cm, n2tf.L / cm, n2tf.SurfaceArea  / m2))

NEXT2.0: radius = 150.0 cm;  drift = 300.0 cm; sensor surface = 7.0685834705770345 m2


#### Xe gas volume in Field Cage: P = 20 bar is equivalent to T = -75 C y P = 0.8 bar (same density)

In [78]:
xeN20 =NN.AngelXe(R=D_N2T_FC/2, L=L_N2T_FC , Rb=1*cm, Cb=1*cm, Ab=1*cm, P=P)

In [79]:
print(xeN20)


        Pressure =  10.00 bar
        Fiducial Radius =  150.00 cm  xe Radius =  151.00 cm
        Fiducial Length =  300.00 cm  Xe Length =  302.00 cm
        
        Fiducial Volume =   21.21 m3 Xe Volume =   21.63 m3
        Fiducial mass = 1221.66 kg Xe mass = 1246.26 kg
        
        Radial buffer radius =    1.00 cm (E/P ~ 2)
        Cathode buffer thickness =    1.00 cm   (E/P~1)
        Anode buffer thickness =    1.00 cm
        
        Long buffer Volume =    0.29 m3
        Long buffer mass =   16.45 kg
        
        Cathode buffer Volume =    0.07 m3
        Cathode buffer mass =    4.13 kg
    
        Anode buffer Volume =    0.07 m3
        Anode buffer mass =    4.13 kg
    
        Total buffer Mass =   24.71 kg
        Cathode Readout Surface =    7.16 m2
    


####  can

In [46]:
t_gas_buffer = 1 * cm

D_N2T_PV = D_N2T_FC  + t_gas_buffer
R_N2T_PV = D_N2T_PV / 2
t_N2T_PV = 5 * cm 
L_N2T_PV = 325 * cm
n2can = shp.Cylinder(R_N2T_PV, L_N2T_PV)

In [48]:
canN20 = NN.AngelCan(Rin=R_N2T_PV, Rout=R_N2T_PV + t_N2T_PV, L=L_N2T_PV, Cp=t_N2T_PV, Ap=t_N2T_PV,
                 cplateType='Flat', aplateType='Flat',
                 FlangeRin=R_N2T_PV, FlangeRout=R_N2T_PV + t_N2T_PV, FlangeT=t_N2T_PV, Mat='Fe')
print(canN20)


        Material = Fe
        Inner Radius =  150.50 cm
        Outer Radius =  155.50 cm
        Length =  325.00 cm
        Cathode plate thickness =    5.00 cm
        Anode plate thickness =    5.00 cm
        Flange thickness =    5.00 cm
        
        Shell Volume =    1.56 m3
        Shell mass = 12294.18 kg
        Shell activity =   4e+08 c/year
        
        Cathode plate Volume =    0.38 m3
        Cathode plate mass = 2989.20 kg
        Cathode plate activity =   9e+07 c/year
        
        Anode plate Volume =    0.38 m3
        Anode plate mass = 2989.20 kg
        Anode plate activity =   9e+07 c/year
        
        Flange Volume =    0.02 m3
        Flange mass =  189.14 kg
        Flange activity =   6e+06 c/year
        
        Can total mass = 18272.58 kg
        Can total activity = 5.66e+08 counts/year
        


#### NAUSICAA EL

In [147]:
EL = NN.AngelEL(EP=1.5*kilovolt/(cm*bar),
                 dV=0.3*kilovolt/cm,
                 P=20*bar,
                 d=5*mm,
                 L=L_NAU_FC / 2,
                 EPmin=1.5*kilovolt/(cm*bar))
print(EL)


        E/P =    1.50 kV * cm^-1* bar^-1
        E/P min =    1.50 kV * cm^-1* bar^-1
        dV = drift voltage =    0.30 kV * cm^-1
        P  = pressure =   20.00 bar
        d  = EL grid gap =    5.00 mm
        L  = drift lenght =   1.50 m
        Grid voltage =   15.00 kV
        Cathode voltage =   60.00 kV
        Yield =  9.40e+02 photons/e
        Yield at EPmin =  9.40e+02 photons/e
        Buffer length =     2.0 cm
        dE/dx =  1.65e+01 electrons/cm
        dG/dx =  1.56e+04 photons/cm
    
        Primary scintillation photons per MeV = 1.3158e+04
        Primary ionization electrons per MeV = 4.0323e+04
        


#### NAUSICAA Energy plane SiPMs

In [148]:
sipm_EP = NN.SiPM(name='Hamamatsu',L=10*mm, QE=0.5,TPB=0.5, AU=3*becquerel*1e-6,ATh=3*becquerel*1e-6)
print(sipm_EP)


        SiPM name = Hamamatsu
        SiPM size =   10.00 mm
        SiPM Surface =  100.00 mm2
        SiPM QE =    0.50
        SiPM TPB eff =    0.50
        SiPM global PDE =    0.25
        U activity =    3.00 muBq
        Th activity =    3.00 muBq
        


#### NAUSICAA Tracking plane SiPMs

In [149]:
sipm_TP = NN.SiPM(name='Hamamatsu',L=1*mm, QE=0.5,TPB=0.5, AU=3*becquerel*1e-7,ATh=3*becquerel*1e-7)
print(sipm_TP)


        SiPM name = Hamamatsu
        SiPM size =    1.00 mm
        SiPM Surface =    1.00 mm2
        SiPM QE =    0.50
        SiPM TPB eff =    0.50
        SiPM global PDE =    0.25
        U activity =    0.30 muBq
        Th activity =    0.30 muBq
        


#### NAUSICAA Tracking plane

In [150]:
TP= NN.AngelSiPM(sipm_TP, EL, Surface=S_NAU, pitch=11*mm, asic=64, pdm=1)
print(TP)


        SiPM QE =    0.50
        SiPM PDE =    0.25
        SiPM size =    1.00 mm
        SiPM Pitch =   11.00 mm
        Linear Packing factor =    0.09
        Detection Surface =     7.1 m2
        SiPM Surface =   0.058 m2
        Coverage =  0.0083
        nof SiPM = 5.8e+04
        nof ASIC = 9.1e+02
        nof PDM = 5.8e+04
        


#### NAUSICAA Energy plane

In [151]:
EP= NN.AngelSiPM(sipm_EP, EL, Surface=S_NAU, pitch=11*mm, asic=64, pdm=25)
print(EP)


        SiPM QE =    0.50
        SiPM PDE =    0.25
        SiPM size =   10.00 mm
        SiPM Pitch =   11.00 mm
        Linear Packing factor =    0.91
        Detection Surface =     7.1 m2
        SiPM Surface =     5.8 m2
        Coverage =    0.83
        nof SiPM = 5.8e+04
        nof ASIC = 9.1e+02
        nof PDM = 2.3e+03
        


In [152]:
new = NN.ANGEL(xeNEW,canNEW,newEP,newEL,newTP)
print(new)


        Optical Gain = 1.05e+03
        Solid Angle  = 2.28e-02
        ScintillationSolidAngle = 2.28e-01
        ELSolidAngle = 2.28e-02
        Scintillation Photons/MeV = 1.32e+04
        Scintillation Reaching Cathode/MeV = 2.99e+03
        Scintillation Photoelectrons/MeV = 1.57e+02
                   EL Photons/MeV = 4.25e+07
                   EL Reaching Cathode/MeV = 9.66e+05
                   EL Photoelectrons/MeV = 2.20e+05
                   EL pe/electron = 1.26e+00

        Background rate (c/(keV*kg*year) = 1.05e-02
        


In [153]:
nau = NN.NausicaAngel(xe,can,EL,EP,TP)
print(nau)


        Optical Gain = 9.40e+02
        Solid Angle  = 4.60e-02
        ScintillationSolidAngle = 4.60e-01
        ELSolidAngle = 4.60e-02
        Scintillation Photons/MeV = 1.32e+04
        Scintillation Reaching Cathode/MeV = 6.06e+03
        Scintillation Photoelectrons/MeV = 1.25e+03
                   EL Photons/MeV = 3.79e+07
                   EL Reaching Cathode/MeV = 1.74e+06
                   EL Photoelectrons/MeV = 3.60e+05
                   EL pe/electron = 8.94e+00

        Background rate (c/(keV*kg*year) = 1.17e-05
        


In [155]:

print(nau.ScintillationReachingCathode(40 * keV))
print(nau.ScintillationPhotoelectrons(40 * keV))
print(nau.ScintillationPhotoelectrons(40 * keV) / nof_esipm)
print(nau.ELReachingCathode(40 * keV))
print(nau.ELPhotoelectrons(40 * keV))
print(nau.ELPhotoelectrons(40 * keV) / nof_esipm)
print(nau.ScintillationPhotoelectrons(2500 * keV))
print(nau.ScintillationPhotoelectrons(2500 * keV) / nof_esipm)
print(nau.ELPhotoelectrons(2500 * keV) /nof_esipm)

Scintillation PES for 2500 keV = 3127.7394893006585
Scintillation PES / SiPM for 2500 keV = 0.053926542918976875
242.21214605144303
50.04383182881054
0.00086282468670363
69772.72465288342
14415.85220100897
0.24854917587946498
3127.7394893006585
0.053926542918976875
15.53432349246656


#### Photons 5 keV

In [ ]:
nau.ScintillationReachingCathode(5*keV)

In [ ]:
nau.ELPhotoelectrons(25*keV)

### Dark Side

In [ ]:
PDM = 2605 * 2
sipm_pdm = 44
sipm = PDM * sipm_pdm
print('number of PDM = {} # sipm/PDM = {} # sipm = {}'.format(
PDM, sipm_pdm, sipm ))

#### BTL

In [ ]:
number_of_cells = 2.5e+5
sipms_per_cell = 1
sipm_btl = number_of_cells * sipms_per_cell
sipm_btl_cost = 5
print('number of cells = {} # sipm/PDM = {} # sipm = {}'.format(
number_of_cells, sipms_per_cell, sipm_btl ))

### Cost SiPMs

In [ ]:
cost_mm2 = 20 / 36
cost_cm2 = 1e+2 * cost_mm2
print(' SiPM: cost/cm2 = {}'.format(cost_cm2))

In [ ]:
nausicaa_ep = 1.1e+5 * cost_cm2 /1e+6
dks = 1.1e+5 * cost_cm2  /1e+6
btl = sipm_btl * sipm_btl_cost  /1e+6
btl2 = number_of_cells * 1.2 * cost_cm2  /1e+6
print('cost of NAUSICAA EP = {}'.format(nausicaa_ep))
print('cost of Dark Side SiPMs = {}'.format(dks))
print('cost of BTL  SiPMs = {}'.format(btl))
print('cost of BTL1  SiPMs = {}'.format(btl2))

In [108]:
90 / (1.3 * 1.3)

53.25443786982248

### Activity calculations

In [ ]:
mat.vessel(1*cm,50*cm,'Cu10')

### Dark current

http://sensl.com/products/c-series/

In [49]:
dark_current_sensl =  30 * kHz/mm2
t0 = 20 # Celsius

In [50]:
def dark_current(t, t0 = 25, dark_current_t0_mm2 = dark_current_sensl):
    rf = (t0 -t) / 10
    return dark_current_t0_mm2 / 2 ** rf

In [51]:
dark_current_N2 = dark_current(t = -90)

In [52]:
dark_current_N2/(hertz/mm2)

10.358009490037318

In [53]:
dark_current_N2/(hertz/cm2)

1035.8009490037316

In [54]:
nof_esipm = 7.2e+3
print('Scintillation PES for 2500 keV = {}'.format(nau.ScintillationPhotoelectrons(2500 * keV)))
print('Scintillation PES / SiPM for 2500 keV = {}'.format(nau.ScintillationPhotoelectrons(2500 * keV) / nof_esipm))
print('Scintillation PES for 40 keV = {}'.format(nau.ScintillationPhotoelectrons(40 * keV)))
print('Scintillation PES / SiPM for 40 keV = {}'.format(nau.ScintillationPhotoelectrons(40 * keV) / nof_esipm))
print('dark current hertz/ cm2 = {}'.format(dark_current_NAUSICAA/(hertz/cm2)))
print('dark current per SiPM in 500 ns  = {}'.format(500E-9*dark_current_NAUSICAA/(hertz/cm2)))
print('dark current per plane in 500 ns  = {}'.format(nof_esipm *500E-9*dark_current_NAUSICAA/(hertz/cm2)))

NameError: name 'nau' is not defined

In [ ]:
dark_current_NAUSICAA/(1e+3*hertz/cm2)

In [ ]:
fig = Image(filename='./FBK_NUV_HD_DCR_T.png')

In [ ]:
fig

In [ ]:
T_lar = -185.85 # in C
T_lxe = -100
T_nau = -75
T_BTL = -30
T0 = 25 # in C
T_K = 273.15

In [ ]:
print('dark current: BTL ={}; N2K = {}; LXe = {}; LAr = {}'.format(
dark_current(t = T_BTL, dark_current_t0_mm2 = 10 * kHz/mm2)/(hertz/mm2), 
dark_current(t = T_nau, dark_current_t0_mm2 = 10 * kHz/mm2)/(hertz/mm2), 
dark_current(t = T_lxe, dark_current_t0_mm2 = 10 * kHz/mm2)/(hertz/mm2),
dark_current(t = T_lar, dark_current_t0_mm2 = 10 * kHz/mm2)/(hertz/mm2)
))

In [ ]:
T = np.arange(-200, 30)

In [ ]:
Tr = 1000 / (T + T_K)

In [ ]:
plt.plot(Tr,T,'b')
plt.grid(True)
plt.xlabel('1000/T (K$^{-1}$)')
plt.ylabel('T (C)')
plt.show()

In [ ]:
1000 / (T_lar + T_K)

In [ ]:
1000 / (T_lxe + T_K)

In [ ]:
1000 / (T_nau + T_K)

In [ ]:
1000 / (T_BTL + T_K)

According to plot, for T = 25 C, DC = 100 kHz while for T = T_lar, DC < 0.01 for low field

Plot is in cps/mm2 

### Liquid argon

In [ ]:
ar_boiling_point = 87.302 # in K K ​(−185.848 °C, ​−302.526 °F)
rho = 1.784 # g/L
rho_l = 1.3954 # g/cm3

m_nausicaa_lar = 13e+6 * rho_l / 1e+6 # in tons


In [ ]:
m_nausicaa_lar

### SiPMs

In [ ]:
ael.IonizationElectrons(10 * keV)

In [ ]:
ael.OpticalGain()

In [ ]:
total_photons_10_keV = ael.OpticalGain() * ael.IonizationElectrons(10 * keV)

In [ ]:
total_photons_10_keV

In [ ]:
geometry_factor = 0.5
total_photons_energy_plane = total_photons_10_keV * geometry_factor * tp.Coverage()

In [ ]:
photons_per_sipm = total_photons_energy_plane / tp.NumberSiPM()

In [ ]:
pes_per_sipm = photons_per_sipm

In [ ]:

cost_mm2 = 20/(6*6)

In [ ]:
cost_mm2

In [ ]:
cost_m2 = cost_mm2 * 1e6

In [ ]:
cost_cm2 = cost_mm2 * 1e4

In [ ]:
cost_cm2